# Summa control notebook

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import copy

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

import summapy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dir_pre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dir_pre = '/Users/karllapo/gdrive/'

# Project specific directories
dirProj = dir_pre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Summa specific directories/names
dirSumma = dirProj + 'summa/'
projName = 'summaTestCases'

# Summa control function

In [3]:
def summaControlFunc(dirModel, siteName, expName, expID, attrFile, datFile, datFileName, allParams, allDecisions):
    summapy.summaFileManager.writeDefault(dirModel, siteName, expName);
    
    summapy.summaFileManager.localAttributesFile(attrFile, dirModel, siteName, expName);

    summapy.summaFileManager.paramTrial(list(allParams.keys()), list(allParams.values()),
                                        dirModel, siteName, expName, expID);

    summapy.summaFileManager.forcingFile(dirModel, siteName, expName, datFileName,
                                         np.array(1001), xrForce=datFile, expID=expID)   ; 
    
    summapy.summaFileManager.decision(allDecisions, dirModel, siteName, expName,
                                      pd.to_datetime(datFile.time[0].values),
                                      pd.to_datetime(datFile.time[-1].values), expID=expID);
    
    summapy.summaFileManager.fileManager(dirModel, '/ModTsfc_summa/', siteName, expName, expID=expID);

In [4]:
def summaRunCommand(expNames, exp, site, fName=None):
    if fName == None:
        fName = 'run' + site + exp + '.sh'
    with open(fName, "w+") as f:
    
        f.write('''#!/bin/bash

# Used to run the test cases for SUMMA

# There are two classes of test cases:
#  1) Test cases based on synthetic/lab data; and
#  2) Test cases based on field data.

# The commands assume that you are in the directory {localInstallation}//summaTestCases/settings/
# and that the control files are in {localInstallation}//summaTestCases/settings/

# Set the SUMMA executable (e.g. /usr/local/bin/summa.exe or wherever you have installed SUMMA)

BASEDIR=`pwd`
SUMMA_EXE="docker run -v ${BASEDIR}:/ModTsfc_summa bartnijssen/summa:docker"


# *************************************************************************************************
''')
        for expID in expNames[exp]:
            fileManagerName = 'summa_fileManager_' + expID + '.txt'
            f.write('${SUMMA_EXE} -p never -s _' + expID
                    +' -m /ModTsfc_summa/settings/'
                    + exp + '/' + site + '/' + fileManagerName)
            f.write('\n')


In [5]:
# ----------
# Forcing data and attributes
siteNames = ['SWA', 'CDP']

os.chdir(dirData)
CDPforce = xr.open_dataset('CDP.ModTsfc.ModelForcing.nc')
SWAforce = xr.open_dataset('SWA.ModTsfc.ModelForcing.nc')
force = {'SWA': SWAforce, 'CDP': CDPforce}

SWAattr = xr.open_dataset('SWA.ModTsfc.Attributes.nc')
CDPattr = xr.open_dataset('CDP.ModTsfc.Attributes.nc')
attr = {'SWA': SWAattr, 'CDP': CDPattr}

# Names of forcing files
datName = {'SWA': 'SWA.ModTsfc.ModelForcing.nc',
           'CDP': 'CDP.ModTsfc.ModelForcing.nc'}


In [6]:
myDecisions = {'astability': 'louisinv',
               'subRouting': 'qInstant',
               'alb_method': 'conDecay'}
allParams = {'albedoMinWinter': 0.6,
             'albedoMinSpring': 0.25,
             'albedoDecayRate': .5*10**6}

dirModel = '/Users/karllapo/gdrive/SnowHydrology/proj/ModTsfc/summa/ModTsfc_summa/'

# ----------
# Experiment names and identifiers
experiments = ['intBias_thermalCond', 'intBias_stability']

# ----------
# Layer
expNames = {'layers': ['layerThin', 'layerThick', 'jrdn1991', 'CLM_2010']}
decisions = {'layerThin': {'snowLayers': 'CLM_2010'},
             'layerThick': {'snowLayers': 'CLM_2010'},
             'CLM_2010': {'snowLayers': 'CLM_2010'},
             'jrdn1991': {'snowLayers': 'jrdn1991'}}
params = {'jrdn1991': None, 'CLM_2010': None}

# UEB-like set-up
layerThin = {'zminLayer3': 100.0000,
             'zminLayer4': 100.0000,
             'zminLayer5': 100.0000,
             'zmaxLayer2_lower': 1000.0000,
             'zmaxLayer3_lower': 1000.0000,
             'zmaxLayer4_lower': 1000.0000,
             'zmaxLayer2_upper': 1000.0000,
             'zmaxLayer3_upper': 1000.0000,
             'zmaxLayer4_upper': 1000.0000}
params.update({'layerThin': layerThin})

# VIC-like set-up
layerThick = {'zminLayer1': 0.0500,
              'zminLayer2': 0.0200,
              'zminLayer3': 100.0000,
              'zminLayer4': 100.0000,
              'zminLayer5': 100.0000,
              'zmaxLayer1_lower': 0.2000,
              'zmaxLayer2_lower': 1000.0000,
              'zmaxLayer3_lower': 1000.0000,
              'zmaxLayer4_lower': 1000.0000,
              'zmaxLayer1_upper': 0.1500,
              'zmaxLayer2_upper': 1000.0000,
              'zmaxLayer3_upper': 1000.0000,
              'zmaxLayer4_upper': 1000.0000}
params.update({'layerThick': layerThick})

# ----------
# Thermal conductivity
expNames['thermalCond'] = ['tyen1965', 'smnv2000_Low', 'smnv2000_Mid', 'smnv2000_High']
params.update({'smnv2000_Mid': {'fixedThermalCond_snow': 0.3500},
               'smnv2000_Low': {'fixedThermalCond_snow': 0.1000},
               'smnv2000_High': {'fixedThermalCond_snow': 1.0000},
               'tyen1965': None})
decisions.update({'smnv2000_Mid': {'thCondSnow' : 'smnv2000'},
                  'smnv2000_Low': {'thCondSnow' : 'smnv2000'},
                  'smnv2000_High': {'thCondSnow' : 'smnv2000'},
                  'tyen1965': {'thCondSnow': 'tyen1965'}})

# ----------
# Stability
expNames['stability'] = ['standard_Low', 'standard_High', 'mahrtexp_Low', 'mahrtexp_High']
params.update({'standard_Low': {'z0Snow': .001}, 'standard_High': {'z0Snow': .1},
               'mahrtexp_Low': {'z0Snow': .001}, 'mahrtexp_High': {'z0Snow': .1}})
decisions.update({'standard_Low': {'astability' : 'standard'},
                  'standard_High': {'astability' : 'standard'},
                  'mahrtexp_Low': {'astability' : 'mahrtexp'},
                  'mahrtexp_High': {'astability' : 'mahrtexp'}})

In [7]:

for s in siteNames:
    for exp in experiments:
        expID_list = []
        if 'stability' in exp:
            for layer in expNames['layers']:
                for stab in expNames['stability']:
                    # Name for identifying files for this experiment
                    expID = layer + '_' + stab
                    expID_list.append(expID)
                    print(expID)

                    # Decisions dictionary
                    inDecisions = myDecisions.copy()
                    inDecisions.update(decisions[layer])
                    inDecisions.update(decisions[stab])

                    # Parameters dictionary
                    inParams = allParams.copy()
                    if not params[layer] is None:
                        inParams.update(params[layer])
                    if not params[stab] is None:
                        inParams.update(params[stab])

                    # Create summa control files
                    summaControlFunc(dirModel, s, exp, expID,
                                     attr[s], force[s], datName[s], inParams, inDecisions)                

        elif 'thermalCond' in exp:
            for layer in expNames['layers']:
                for thCond in expNames['thermalCond']:
                    # Name for identifying files for this experiment
                    expID = layer + '_' + thCond
                    expID_list.append(expID)

                    # Decisions dictionary
                    inDecisions = myDecisions.copy()
                    inDecisions.update(decisions[layer])
                    inDecisions.update(decisions[thCond])

                    # Parameters dictionary
                    inParams = allParams.copy()
                    if not params[layer] is None:
                        inParams.update(params[layer])
                    if not params[thCond] is None:
                        inParams.update(params[thCond])

                    # Create summa control files
                    summaControlFunc(dirModel, s, exp, expID,
                                     attr[s], force[s], datName[s], inParams, inDecisions)     

        # Write the shell script for running this experiment
        os.chdir(dirModel)
        summaRunCommand({exp: expID_list}, exp, s)
                

500000.0    1000.0    1000.0    1000.0    1000.0    0.25    1000.0    100.0    1000.0    100.0    100.0    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
500000.0    1000.0    1000.0    1000.0    1000.0    0.25    1000.0    100.0    0.1    1000.0    100.0    100.0    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
500000.0    1000.0    1000.0    1000.0    1000.0    0.25    1000.0    100.0    0.35    1000.0    100.0    100.0    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
500000.0    1000.0    1000.0    1000.0    1000.0    0.25    1000.0    100.0    1.0    1000.0    100.0    100.0    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
500000.0    1000.0    0.05    100.0    1000.0    0.02    100.0    1000

In [8]:
print(expID_list)

['layerThin_standard_Low', 'layerThin_standard_High', 'layerThin_mahrtexp_Low', 'layerThin_mahrtexp_High', 'layerThick_standard_Low', 'layerThick_standard_High', 'layerThick_mahrtexp_Low', 'layerThick_mahrtexp_High', 'jrdn1991_standard_Low', 'jrdn1991_standard_High', 'jrdn1991_mahrtexp_Low', 'jrdn1991_mahrtexp_High', 'CLM_2010_standard_Low', 'CLM_2010_standard_High', 'CLM_2010_mahrtexp_Low', 'CLM_2010_mahrtexp_High']
